In [5]:
from custom_env import CustomEnv
from simulation_manager import SimulationManager
PYGAME_DETECT_AVX2=1

In [6]:
model_args = {
    'num_actions': 11
}

simulation_manager_args = {
    'number_of_environments': 10,
    'number_of_curricula': 3
}

game_manager_args = {
    'map_pixel_size': 16,
    'screen_size': 800,
    'kg_completness': 1,
    'vision_range': 2
}

In [7]:
from game_manager import GameManager
game_manager = GameManager(game_manager_args['map_pixel_size'], game_manager_args['screen_size'],
                            game_manager_args['kg_completness'], game_manager_args['vision_range'])
game_manager.run()

Player node index: 0, player position: (0, 11)


KeyboardInterrupt: 

In [1]:
from simulation_manager import SimulationManager
simulation_manager = SimulationManager(game_manager_args,
                                        simulation_manager_args['number_of_environments'],
                                        simulation_manager_args['number_of_curricula'])

simulation_manager.game_managers[0].run()

<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.5.2 (SDL 2.28.5, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'game_manager_args' is not defined